<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/Langchain%2BELYZA_japanese_Llama_2_7b_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate bitsandbytes sentence_transformers
!pip install -q langchain pypdf tiktoken faiss-cpu
# faiss-gpu はGPUの節約wのために使わない。

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# LLM
tokenizer = AutoTokenizer.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct"
)
model = AutoModelForCausalLM.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# MIT License
# https://gist.github.com/alfredplpl
# https://gist.github.com/alfredplpl/57a6338bce8a00de9c9d95bbf1a6d06d

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate


# Prompt
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = (
    "以下の英文で記述された論文の一部をよく読み、"
    "ユーザーからの質問に日本語で答えてください。"
)
text = "{context}\nユーザからの質問は次のとおりです。{question}"
template = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
    prompt=text,
    e_inst=E_INST,
)

# load pdf
loader = PyPDFLoader("https://arxiv.org/pdf/2303.13495.pdf")
documents = loader.load()

#
print(documents)

# split text
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size=50,
    chunk_overlap=0
)
texts = text_splitter.split_documents(documents)


# embeddings
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large",
                                   model_kwargs = {'device': 'cpu'}
                                   )

db = FAISS.from_documents(texts, embeddings)

# Retrival
retriever = db.as_retriever(search_kwargs={"k": 3})
# model = AutoModelForCausalLM.from_pretrained(model_id,load_in_8bit=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
PROMPT = PromptTemplate(template=template,
                        input_variables=["question","context"],
                        template_format="f-string"
                        )
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(
    llm=HuggingFacePipeline(pipeline=pipe),
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=False,
    chain_type_kwargs=chain_type_kwargs)

# Run :)
result = qa.run("この論文の学術的貢献を要約しなさい。")
print(result)

 学術的貢献は以下の通りです。

- 新しい問題、関係逆転、を学び、関係の導き出しから関係のプリセットを学習する手法を提案しました。
- 関係の導き出しと関係のプリセットの両者が必要であることを詳細な実験から示しました。
- これらの手法は、関係の導き出しと関係のプリセットを学習するReVerseフレームワークを提案しました。
- これらの手法は、関係の導き出しと関係のプリセットの両者が重要であることを示す詳細な実験を行いました。
- これらの手法は、関係の導き出しと関係のプリセットを学習するReVerseフレームワークを提案しました。

これらの手法は、関係の導き出しと関係のプリセットの両者が重要であることを示す詳細な実験を行いました。
 与えられた論文の結論を要約いたします。

論文は、動物の名前を収録したデータセットを用いて、動物の名前を聞いた際に人がどのように応じるかを研究しています。その結果、動物の種類によって人の応答が異なることが明らかとなりました。この研究の結果は、動物の種類を識別するための新たな手法の開発に役立つ可能性があります。


In [ ]:
%%time
result = qa.run("この論文の結論を要約しなさい。")
print(result)

 与えられた論文の結論を要約いたします。

論文は、動物の名前を収録したデータセットを用いて、動物の名前を聞いた際に人がどのように応じるかを研究しています。その結果、動物の種類によって人の応答が異なることが明らかとなりました。この研究の結果は、動物の種類を識別するための新たな手法の開発に役立つ可能性があります。


In [ ]:
%%time
result = qa.run("この論文ではどんなデータセットを用いましたか？")
print(result)

 与えられた論文の一部をよく読み、ユーザーからの質問に回答いたします。

論文では、データセット「t-SNE」を用いています。データセットは以下のリンクから入手することができます。

http://www.jmlr.org/papers/volume9/villegas08a/villegas08a.zip

また、データセット「bilistic models」は以下のリンクから入手することができます。

https://arxiv.org/abs/2112.00390
CPU times: user 12 s, sys: 40.3 ms, total: 12 s
Wall time: 17.9 s


In [ ]:
%%time
result = qa.run("この論文のPreposition Priorとは？")
print(result)

 承知しました。ユーザーからの質問に回答いたします。

Preposition Priorとは、前置詞のリストです。このリストを元に、関係補完の際に関係補完の候補を生成することができるというものです。
CPU times: user 7.09 s, sys: 22.9 ms, total: 7.11 s
Wall time: 9.61 s


In [ ]:
%%time
result = qa.run("この論文のBenchmarkの結果をまとめてください。")
print(result)

 承知しました。

Benchmarkの結果として、以下のような情報が得られました。

- ステアリングとノイズ除去の各モデルの性能を比較した結果、ステアリングのモデルの方がノイズ除去のモデルよりも性能が良いことがわかりました。
- ステアリングとノイズ除去の各モデルの性能を、異なるパラメータの設定で評価した結果、パラメータの設定によって性能が大きく変わることがわかりました。

これらの結果は、ReVersion Benchmarkの詳細について記述された英文の一部であり、全文は以下のリンクからご覧ください。

[リンク]
CPU times: user 16.6 s, sys: 70 ms, total: 16.7 s
Wall time: 16.9 s


In [ ]:
%%time
result = qa.run("この論文を易しくまとめてください。")
print(result)

 与えられた論文を易しくまとめます。

この論文は、画像認識のためのディープラーニングの手法について述べています。

まず、画像を認識するためには、画像を特徴量に変換する必要があります。この特徴量の変換を行うのが、「コースディスクリプション」と「ノイズレベル」の2つのモジュールです。

次に、特徴量に変換した画像を、ディープラーニングのニューラルネットワークに入力します。この入力画像を、適切なサイズに括ります。これを「サンプリング」と呼びます。

最後に、ニューラルネットワークが学習した後、順を追って画像を認識します。

以上が、この論文の要点です。
CPU times: user 20 s, sys: 93.8 ms, total: 20.1 s
Wall time: 29.2 s


In [ ]:
%%time
result = qa.run("この論文の主張を詳細にまとめてください。")
print(result)

 与えられた論文の主張を詳細にまとめます。

論文は、犬や猿などの動物を用いて、画像認識の研究を行っています。その際、犬や猿はそれぞれ「犬<R>犬」や「猿<R>猿」という関係性を持っていることがわかります。また、人や木などの「コースディスクリプション」を持っていることもわかります。これらの情報を用いて、画像認識のアルゴリズムを改良することで、より正確な認識を行うことができることが示されています。
CPU times: user 15.5 s, sys: 64.9 ms, total: 15.6 s
Wall time: 18.3 s
